In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import gzip
import sys
import os

In [6]:
import mediator_was.processing.gtex as gtex
import mediator_was.processing.predixcan as predixcan
import yaml

In [7]:
config = yaml.load(open('../../config.yaml'))

In [8]:
config

{'data': {'gtex': {'dir': '/projects/gtex/data',
   'files': {'expression': 'ExpressionFiles/gtex.portal.normalized/Whole_Blood_Analysis.expr.txt',
    'genotypes': 'GenotypeFiles/midpoint/GTEx_Analysis_20150112_OMNI_2.5M_5M_450Indiv_chr1to22_phased_genot_imput_info04_maf01_HWEp1E6_ConstrVarIDs.vcf.gz'}},
  'predixcan': {'dir': '/home/kbhutani/mediator/data/predixcan/',
   'files': {'database': 'DGN-WB_0.5.txt'}},
  'references': {'dbsnp': '/projects/references/snp142Common.txt.gz'}}}

## Model: Alpha-1

#### Process and load Gustavo Data

In [9]:
# Add positions to database
new_db = os.path.join(predixcan.main_dir, "EN.withpos.tsv")
#predixcan.add_positions_to_database(fn=new_db)

In [25]:
# Load database
en_df = predixcan.load_database(new_db, alpha=1)

In [26]:
en_df.head()

,gene,rsid,refAllele,alt,beta,alpha,id,chr,start,end,chromosome,position
0,A2LD1,rs11069419,T,C,-0.034054,1,1359,chr13,101561096,101561097,chr13,101561097
1,A2LD1,rs11842969,C,T,-0.381005,1,1357,chr13,101214374,101214375,chr13,101214375
2,A2LD1,rs1283142,C,A,-0.013816,1,1358,chr13,101392263,101392264,chr13,101392264
3,A2LD1,rs1283211,C,T,0.022643,1,1358,chr13,101369270,101369271,chr13,101369271
4,TMTC4,rs1283211,C,T,0.006821,1,1358,chr13,101369270,101369271,chr13,101369271


### Predicted Values

#### TODO: Unit Test for Predicted Values

In [76]:
genotype_file = gtex.genotype_fn
predictions_df, not_found, incorrect, switched = predixcan.predict_expression(en_df, genotype_file)

Total 224082
Not Found 8
Incorrect 0
Switched 0


In [95]:
pd.concat([pd.DataFrame(x).T for x in not_found])

,gene,rsid,refAllele,alt,beta,alpha,id,chr,start,end,chromosome,position
21285,HHIPL2,rs34651530,A,G,-0.04849929,1,2286,chr1,222985196,222985197,1,222985197
84491,ZP3,rs17149187,A,G,0.03322222,1,1164,chr7,75998572,75998573,7,75998573
127715,MED17,rs11020719,G,A,-0.0413322,1,1302,chr11,94008519,94008520,11,94008520
138075,KRR1,rs1026784,G,A,-0.04398848,1,1164,chr12,75997732,75997733,12,75997733
151878,FCF1,rs17183663,C,A,0.002284925,1,1164,chr14,76001456,76001457,14,76001457
170927,ZNF469,rs4843756,C,T,0.03219198,1,1256,chr16,88003483,88003484,16,88003484
183491,CD300C,rs3744203,A,G,-0.01866636,1,1141,chr17,73000060,73000061,17,73000061
183492,NT5C,rs3744203,A,G,-0.006345411,1,1141,chr17,73000060,73000061,17,73000061


#### Note that it did not compute on the HLA related genes. Let's remove them from further analysis.

In [80]:
hla_related_genes = set(en_df[en_df['chromosome'] == 25]['gene'])

In [91]:
full_predicted_genes = set(predictions_df.index) - hla_related_genes
full_predicted_df = predictions_df.ix[full_predicted_genes]
full_predicted_df.columns = full_predicted_df.columns.map(lambda x: "-".join(x.split('-')[:2]))

In [93]:
full_predicted_df.head()

,GTEX-P4PP,GTEX-PWO3,GTEX-SNOS,GTEX-PW2O,GTEX-RU72,GTEX-VJYA,GTEX-U3ZH,GTEX-PLZ4,GTEX-P44G,GTEX-WOFL,...,GTEX-YF7O,GTEX-11TTK,GTEX-Y3IK,GTEX-12WSE,GTEX-1212Z,GTEX-YEC3,GTEX-11ONC,GTEX-ZZPT,GTEX-ZVTK,GTEX-XBEC
UBE2Q1,-0.121079,-0.066371,0.135165,0.043322,-0.167153,0.059409,-0.064592,0.117333,0.053394,-0.029087,...,0.118536,0.034042,0.078485,-0.042171,0.019912,-0.001645,0.131741,0.083236,0.007399,-0.054157
RNF14,-0.344138,-0.144490,-0.348426,0.056915,-0.163442,-0.328642,-0.360858,-0.360858,0.043541,-0.165200,...,-0.159453,-0.163442,-0.137155,-0.174767,-0.174767,0.026638,-0.339966,-0.159804,-0.328642,-0.142551
UBE2Q2,0.185559,0.194231,-0.018047,0.290495,0.092135,-0.163189,0.311871,0.441576,0.089022,0.124410,...,0.226836,0.209874,-0.072307,-0.020016,0.147939,0.084239,0.293504,0.169062,0.191851,0.216087
RNF10,-0.032507,-0.068161,-0.056125,-0.047314,0.041432,-0.046286,0.013369,-0.038442,-0.034058,-0.020994,...,-0.026834,-0.049426,0.045150,0.031421,-0.047524,-0.007546,0.034918,-0.029033,-0.029078,-0.063987
RNF13,0.706117,0.000000,0.353058,0.000000,0.000000,0.044834,0.000000,0.000000,0.000000,0.353058,...,0.000000,0.353058,0.397892,0.000000,0.000000,0.353058,0.353058,0.000000,0.000000,0.000000


In [98]:
full_predicted_df.to_csv('../data/predixcan/predictions.minus-hla.tsv',
                        sep="\t")

## Model Alpha=0.5

In [29]:
# Load database
en_df = predixcan.load_database(new_db, alpha=0.5)
genotype_file = gtex.genotype_fn
predictions_df, not_found, incorrect, switched = predixcan.predict_expression(en_df, genotype_file)

Total 331599
Not Found 13
Incorrect 0
Switched 0


In [30]:
pd.concat([pd.DataFrame(x).T for x in not_found])

,gene,rsid,refAllele,alt,beta,alpha,id,chr,start,end,chromosome,position
15588,CDC7,rs10801817,C,T,-0.002368292,0.5,1279,chr1,91001088,91001089,1,91001089
32141,HHIPL2,rs34651530,A,G,-0.05389286,0.5,2286,chr1,222985196,222985197,1,222985197
66724,C3orf52,rs10934162,C,T,0.005354781,0.5,1439,chr3,112001324,112001325,3,112001325
124752,CDK13,rs2329611,C,T,0.003770901,0.5,890,chr7,40008948,40008949,7,40008949
127761,ZP3,rs17149187,A,G,0.03886865,0.5,1164,chr7,75998572,75998573,7,75998573
193217,MED17,rs11020719,G,A,-0.01918575,0.5,1302,chr11,94008519,94008520,11,94008520
209529,KRR1,rs1026784,G,A,-0.03149808,0.5,1164,chr12,75997732,75997733,12,75997733
221980,CDC16,rs7994556,C,T,0.03752229,0.5,1462,chr13,114998075,114998076,13,114998076
229195,FCF1,rs17183663,C,A,0.001574773,0.5,1164,chr14,76001456,76001457,14,76001457
256578,ZNF469,rs4843756,C,T,0.01060286,0.5,1256,chr16,88003483,88003484,16,88003484


In [31]:
hla_related_genes = set(en_df[en_df['chromosome'] == 25]['gene'])
full_predicted_genes = set(predictions_df.index) - hla_related_genes
full_predicted_df = predictions_df.ix[full_predicted_genes]
full_predicted_df.columns = full_predicted_df.columns.map(lambda x: "-".join(x.split('-')[:2]))

In [33]:
full_predicted_df.to_csv('../../data/predixcan/predictions.minus-hla.alpha0.5.tsv',
                        sep="\t")